In [3]:
import pandas as pd

#### Vérification du fichier

In [ ]:
filename = "economicref-france-sirene-v3.csv"
with open(filename, "rb") as f:
    print(f.read(100))

b"\xef\xbb\xbfSIREN;NIC;SIRET;Statut de diffusion de l'\xc3\xa9tablissement;Date de cr\xc3\xa9ation de l'\xc3\xa9tablissement;Tra"


---

#### Stats

In [5]:
# Visualisation des données
df=pd.read_csv(filename, sep=";")
pd.set_option('display.max_columns', None)
df.head()

/tmp/ipykernel_21336/2077608544.py:2: DtypeWarning: Columns (17,43,44,74,75) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(filename, sep=";")


,SIREN,NIC,SIRET,Statut de diffusion de l'établissement,Date de création de l'établissement,Tranche de l'effectif de l'établissement,Tranche de l'effectif de l'établissement triable,Année de la tranche d'effectif de l'établissement,Activité principale de l'établissement,Date de la dernière mise à jour de l'établissement,Etablissement siège,Nombre de periodes de l'établissement,Complément d'adresse de l'établissement,Numéro de voie de l'établissement,Indice de répétition de l'établissement,Type de voie de l'établissement,Libellé de la voie de l'établissement,Code postal de l'établissement,Commune de l'établissement,Libellé de la commune de l'établissement à l'étranger,Distribution spéciale de l'établissement,Code commune de l'établissement,Code cedex de l'établissement,Libellé cedex de l'établissement,Code du pays de l'établissement étranger,Libellé du pays de l'établissement étranger,Complément d'adresse de l'établissement 2,Numero de voie de l'établissement 2,Indice de répétition de l'établissement 2,Type de voie de l'établissement 2,Libellé de la voie de l'établissement 2,Code postal de l'établissement 2,Libellé de la commune de l'établissement 2,Libellé de la commune de l'établissement étranger 2,Distribution spéciale de l'établissement 2,Code de la commune de l'établissement 2,Code cedex de l'établissement 2,Libellé cedex de l'établissement 2,Code pays de l'établissement,Libellé du pays de l'établissement étranger.1,Date du début de la période de l'établissement,Etat administratif de l'établissement,Enseigne de l'établissement 1,Enseigne de l'établissement 2,Enseigne de l'établissement 3,Dénomination usuelle de l'établissement,Activité principale de l'établissement.1,Nomenclature principale de l'établissement,Caractère employeur de l'établissement,Statut de diffusion de l'unité légale,Unité légale purgée,Date de création de l'unité légale,Sigle de l'unité légale,Civilité de la personne physique,Prénom de la personne physique 1,Prénom de la personne physique 2,Prénom de la personne physique 3,Prénom de la personne physique 4,Prénom usuel de la personne physique,Pseudonyme de la personne physique,Identifiant association de l'unité légale,Tranche de l'effectif de l'unité légale,Tranche de l'effectif de l'unité légale triable,Année de la tranche de l'effectif de l'unité légale,Date du dernier traitement de l'unité légale,Nombre de périodes de l'unité légale,Catégorie de l'entreprise,Année de la catégorie de l'entreprise,Date de début de l'unité légale,Etat administratif de l'unité légale,Nom de la personne physique,Nom d'usage de la personne physique,Dénomination de l'unité légale,Dénomination usuelle de l'unité légale 1,Dénomination usuelle de l'unité légale 2,Dénomination usuelle de l'unité légale 3,Catégorie juridique de l'unité légale,Activité principale de l'unité légale,Nomenclature de l'activité principale de l'unité légale,NIC du siège de l'unité légale,Economie sociale et solidaire unité légale,Société à mission unité légale,Caractère employeur de l'unité légale,Code EPCI de l'établissement,Libellé de l'EPCI de l'établissement,Code de l'arrondissement de l'établissement,Code du département de l'établissement,Département de l'établissement,Code de la région de l'établissement,Région de l'établissement,Section de l'établissement,Sous-section de l'établissement,Division de l'établissement,Groupe de l'établissement,Classe de l'établissement,Section de l'unité légale,Sous-section de l'unité légale,Division de l'unité légale,Groupe de l'unité légale,Classe de l'unité légale,Nature juridique de l'unité légale,Première ligne de l'adressage,Adresse de l'établissement,SIRET du siège de l'unité légale,Date de fermeture de l'établissement,Date de fermeture de l'unité légale,Géolocalisation de l'établissement
0,894478809,11,89447880900011,O,2021-02-01,Etablissement non employeur,-1,NaN,NaN,2024-05-08T07:06:49+02:00,oui,2,NaN,NaN,NaN,NaN,36 FLURWEG,NaN,NaN,91623 SACHSEN BEI ANSBACH,NaN,NaN,NaN,NaN,99109.0,ALLEMAGNE,NaN,NaN,NaN,NaN,

In [6]:
print("Number of rows : {}".format(df.shape[0]))
print()

print("Basics statistics: ")
df_desc = df.describe(include="all")
display(df_desc)
print()

print("Total and percentage of missing values: ")
missing_total = df.isnull().sum().sum()
display(missing_total)
missing_percent = 100 * df.isnull().sum() / df.shape[0]
display(missing_percent[missing_percent != 0])

Number of rows : 1174033

Basics statistics: 


,SIREN,NIC,SIRET,Statut de diffusion de l'établissement,Date de création de l'établissement,Tranche de l'effectif de l'établissement,Tranche de l'effectif de l'établissement triable,Année de la tranche d'effectif de l'établissement,Activité principale de l'établissement,Date de la dernière mise à jour de l'établissement,Etablissement siège,Nombre de periodes de l'établissement,Complément d'adresse de l'établissement,Numéro de voie de l'établissement,Indice de répétition de l'établissement,Type de voie de l'établissement,Libellé de la voie de l'établissement,Code postal de l'établissement,Commune de l'établissement,Libellé de la commune de l'établissement à l'étranger,Distribution spéciale de l'établissement,Code commune de l'établissement,Code cedex de l'établissement,Libellé cedex de l'établissement,Code du pays de l'établissement étranger,Libellé du pays de l'établissement étranger,Complément d'adresse de l'établissement 2,Numero de voie de l'établissement 2,Indice de répétition de l'établissement 2,Type de voie de l'établissement 2,Libellé de la voie de l'établissement 2,Code postal de l'établissement 2,Libellé de la commune de l'établissement 2,Libellé de la commune de l'établissement étranger 2,Distribution spéciale de l'établissement 2,Code de la commune de l'établissement 2,Code cedex de l'établissement 2,Libellé cedex de l'établissement 2,Code pays de l'établissement,Libellé du pays de l'établissement étranger.1,Date du début de la période de l'établissement,Etat administratif de l'établissement,Enseigne de l'établissement 1,Enseigne de l'établissement 2,Enseigne de l'établissement 3,Dénomination usuelle de l'établissement,Activité principale de l'établissement.1,Nomenclature principale de l'établissement,Caractère employeur de l'établissement,Statut de diffusion de l'unité légale,Unité légale purgée,Date de création de l'unité légale,Sigle de l'unité légale,Civilité de la personne physique,Prénom de la personne physique 1,Prénom de la personne physique 2,Prénom de la personne physique 3,Prénom de la personne physique 4,Prénom usuel de la personne physique,Pseudonyme de la personne physique,Identifiant association de l'unité légale,Tranche de l'effectif de l'unité légale,Tranche de l'effectif de l'unité légale triable,Année de la tranche de l'effectif de l'unité légale,Date du dernier traitement de l'unité légale,Nombre de périodes de l'unité légale,Catégorie de l'entreprise,Année de la catégorie de l'entreprise,Date de début de l'unité légale,Etat administratif de l'unité légale,Nom de la personne physique,Nom d'usage de la personne physique,Dénomination de l'unité légale,Dénomination usuelle de l'unité légale 1,Dénomination usuelle de l'unité légale 2,Dénomination usuelle de l'unité légale 3,Catégorie juridique de l'unité légale,Activité principale de l'unité légale,Nomenclature de l'activité principale de l'unité légale,NIC du siège de l'unité légale,Economie sociale et solidaire unité légale,Société à mission unité légale,Caractère employeur de l'unité légale,Code EPCI de l'établissement,Libellé de l'EPCI de l'établissement,Code de l'arrondissement de l'établissement,Code du département de l'établissement,Département de l'établissement,Code de la région de l'établissement,Région de l'établissement,Section de l'établissement,Sous-section de l'établissement,Division de l'établissement,Groupe de l'établissement,Classe de l'établissement,Section de l'unité légale,Sous-section de l'unité légale,Division de l'unité légale,Groupe de l'unité légale,Classe de l'unité légale,Nature juridique de l'unité légale,Première ligne de l'adressage,Adresse de l'établissement,SIRET du siège de l'unité légale,Date de fermeture de l'établissement,Date de fermeture de l'unité légale,Géolocalisation de l'établissement
count,1.174033e+06,1.174033e+06,1.174033e+06,1174033,1174033,1174033,1.174033e+06,122310.0,230160,1174033,1174033,1.174033e+06,239393,1.034997e+06,54637,1063726,1163824,1139821.0,1140331,33408,0.0,1140331,0.0,0.0,33701.000000


Total and percentage of missing values: 


np.int64(52365694)

Année de la tranche d'effectif de l'établissement    89.582065
Activité principale de l'établissement               80.395781
Complément d'adresse de l'établissement              79.609347
Numéro de voie de l'établissement                    11.842597
Indice de répétition de l'établissement              95.346213
                                                       ...    
Première ligne de l'adressage                        44.519277
Adresse de l'établissement                            0.860879
Date de fermeture de l'établissement                  1.827291
Date de fermeture de l'unité légale                   9.586783
Géolocalisation de l'établissement                    2.529230
Length: 68, dtype: float64

In [7]:
df.columns.to_list()

['SIREN',
 'NIC',
 'SIRET',
 "Statut de diffusion de l'établissement",
 "Date de création de l'établissement",
 "Tranche de l'effectif de l'établissement",
 "Tranche de l'effectif de l'établissement triable",
 "Année de la tranche d'effectif de l'établissement",
 "Activité principale de l'établissement",
 "Date de la dernière mise à jour de l'établissement",
 'Etablissement siège',
 "Nombre de periodes de l'établissement",
 "Complément d'adresse de l'établissement",
 "Numéro de voie de l'établissement",
 "Indice de répétition de l'établissement",
 "Type de voie de l'établissement",
 "Libellé de la voie de l'établissement",
 "Code postal de l'établissement",
 "Commune de l'établissement",
 "Libellé de la commune de l'établissement à l'étranger",
 "Distribution spéciale de l'établissement",
 "Code commune de l'établissement",
 "Code cedex de l'établissement",
 "Libellé cedex de l'établissement",
 "Code du pays de l'établissement étranger",
 "Libellé du pays de l'établissement étranger",


In [8]:
df['Nature juridique de l\'unité légale'].value_counts()

Nature juridique de l'unité légale
Entrepreneur individuel                                                            651606
SAS, société par actions simplifiée                                                228589
Société à responsabilité limitée (sans autre indication)                           210585
Société étrangère non immatriculée au RCS                                           31530
Société civile immobilière                                                           8729
                                                                                    ...  
Autre SA coopérative à directoire                                                       1
Centre technique industriel ou comité professionnel du développement économique         1
Régime général de la Sécurité Sociale                                                   1
SARL coopérative de consommation                                                        1
Autorité administrative ou publique indépendante                 

In [9]:
a_exclure = [
    "Entrepreneur individuel",
    "Société civile immobilière",
    "Société civile immobilière de construction-vente",
    "Société étrangère non immatriculée au RCS",
    "Association déclarée",
]

df = df[~df["Nature juridique de l'unité légale"].isin(a_exclure)]

In [10]:
pd.set_option("display.max_rows", None)
df["Nature juridique de l'unité légale"].value_counts()

Nature juridique de l'unité légale
SAS, société par actions simplifiée                                                                                                              228589
Société à responsabilité limitée (sans autre indication)                                                                                         210585
Société en nom collectif                                                                                                                           8712
Autre société civile                                                                                                                               5232
Société civile de moyens                                                                                                                           2974
Société d'exercice libéral à responsabilité limitée                                                                                                2626
Indivision entre personnes physiques                 

In [11]:
df.shape

(472301, 107)

In [12]:
# SAS classique exacte
mask_sas_classique = df["Nature juridique de l'unité légale"] == "SAS, société par actions simplifiée"

# SARL exacte
mask_sarl_exacte = df["Nature juridique de l'unité légale"] == "Société à responsabilité limitée (sans autre indication)"

# Conserver uniquement ces deux formes
df_filtered = df[mask_sas_classique | mask_sarl_exacte]

# Vérification
print(df_filtered["Nature juridique de l'unité légale"].value_counts())

Nature juridique de l'unité légale
SAS, société par actions simplifiée                         228589
Société à responsabilité limitée (sans autre indication)    210585
Name: count, dtype: int64


In [13]:
df_filtered.shape

(439174, 107)

In [14]:
df_filtered.head()

,SIREN,NIC,SIRET,Statut de diffusion de l'établissement,Date de création de l'établissement,Tranche de l'effectif de l'établissement,Tranche de l'effectif de l'établissement triable,Année de la tranche d'effectif de l'établissement,Activité principale de l'établissement,Date de la dernière mise à jour de l'établissement,Etablissement siège,Nombre de periodes de l'établissement,Complément d'adresse de l'établissement,Numéro de voie de l'établissement,Indice de répétition de l'établissement,Type de voie de l'établissement,Libellé de la voie de l'établissement,Code postal de l'établissement,Commune de l'établissement,Libellé de la commune de l'établissement à l'étranger,Distribution spéciale de l'établissement,Code commune de l'établissement,Code cedex de l'établissement,Libellé cedex de l'établissement,Code du pays de l'établissement étranger,Libellé du pays de l'établissement étranger,Complément d'adresse de l'établissement 2,Numero de voie de l'établissement 2,Indice de répétition de l'établissement 2,Type de voie de l'établissement 2,Libellé de la voie de l'établissement 2,Code postal de l'établissement 2,Libellé de la commune de l'établissement 2,Libellé de la commune de l'établissement étranger 2,Distribution spéciale de l'établissement 2,Code de la commune de l'établissement 2,Code cedex de l'établissement 2,Libellé cedex de l'établissement 2,Code pays de l'établissement,Libellé du pays de l'établissement étranger.1,Date du début de la période de l'établissement,Etat administratif de l'établissement,Enseigne de l'établissement 1,Enseigne de l'établissement 2,Enseigne de l'établissement 3,Dénomination usuelle de l'établissement,Activité principale de l'établissement.1,Nomenclature principale de l'établissement,Caractère employeur de l'établissement,Statut de diffusion de l'unité légale,Unité légale purgée,Date de création de l'unité légale,Sigle de l'unité légale,Civilité de la personne physique,Prénom de la personne physique 1,Prénom de la personne physique 2,Prénom de la personne physique 3,Prénom de la personne physique 4,Prénom usuel de la personne physique,Pseudonyme de la personne physique,Identifiant association de l'unité légale,Tranche de l'effectif de l'unité légale,Tranche de l'effectif de l'unité légale triable,Année de la tranche de l'effectif de l'unité légale,Date du dernier traitement de l'unité légale,Nombre de périodes de l'unité légale,Catégorie de l'entreprise,Année de la catégorie de l'entreprise,Date de début de l'unité légale,Etat administratif de l'unité légale,Nom de la personne physique,Nom d'usage de la personne physique,Dénomination de l'unité légale,Dénomination usuelle de l'unité légale 1,Dénomination usuelle de l'unité légale 2,Dénomination usuelle de l'unité légale 3,Catégorie juridique de l'unité légale,Activité principale de l'unité légale,Nomenclature de l'activité principale de l'unité légale,NIC du siège de l'unité légale,Economie sociale et solidaire unité légale,Société à mission unité légale,Caractère employeur de l'unité légale,Code EPCI de l'établissement,Libellé de l'EPCI de l'établissement,Code de l'arrondissement de l'établissement,Code du département de l'établissement,Département de l'établissement,Code de la région de l'établissement,Région de l'établissement,Section de l'établissement,Sous-section de l'établissement,Division de l'établissement,Groupe de l'établissement,Classe de l'établissement,Section de l'unité légale,Sous-section de l'unité légale,Division de l'unité légale,Groupe de l'unité légale,Classe de l'unité légale,Nature juridique de l'unité légale,Première ligne de l'adressage,Adresse de l'établissement,SIRET du siège de l'unité légale,Date de fermeture de l'établissement,Date de fermeture de l'unité légale,Géolocalisation de l'établissement
3,384210365,35,38421036500035,O,2020-06-01,0 salarié,0,2023.0,NaN,2025-02-14T03:16:11+01:00,oui,2,NaN,33.0,NaN,Avenue,CLAUDE DEBUSSY,92110,CLICHY,NaN,NaN,92024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

---

In [16]:
df_filtered.dtypes

SIREN                                                        int64
NIC                                                          int64
SIRET                                                        int64
Statut de diffusion de l'établissement                      object
Date de création de l'établissement                         object
Tranche de l'effectif de l'établissement                    object
Tranche de l'effectif de l'établissement triable             int64
Année de la tranche d'effectif de l'établissement          float64
Activité principale de l'établissement                      object
Date de la dernière mise à jour de l'établissement          object
Etablissement siège                                         object
Nombre de periodes de l'établissement                        int64
Complément d'adresse de l'établissement                     object
Numéro de voie de l'établissement                          float64
Indice de répétition de l'établissement                     ob

##### Extraction du dataset en Parquet

In [ ]:
# 1. On crée une copie propre pour isoler df_filtered
df_final = df_filtered.copy()

# 2. On traite les colonnes problématiques

for col in df_final.columns:
    if df_final[col].dtype == 'object':
        df_final[col] = df_final[col].fillna('').astype(str)

# 3. On force aussi les colonnes de codes (souvent en float à cause des NaN)
cols_codes = [
    "Code postal de l'établissement", 
    "Code commune de l'établissement",
    "Code du pays de l'établissement étranger",
    "Code EPCI de l'établissement"
]

for col in cols_codes:
    if col in df_final.columns:
        df_final[col] = df_final[col].fillna('').astype(str)

# 4. Exportation finale en Parquet
df_final.to_parquet('sas_sarl_filtered.parquet', index=False, engine='pyarrow')

print("Succès ! Le fichier est prêt pour l'EDA.")

Succès ! Le fichier est prêt pour l'EDA.


In [ ]:
# df_filtered.to_csv('sas_sarl_filtered.csv', index=False, encoding="utf-8")